In [1]:
##Importing dataset and splitting data

#Importing Libraries
import pandas as pd
from sklearn.model_selection import train_test_split

#Importing csv file
df = pd.read_csv('Tagged_Mixed.csv')

In [2]:
import pandas as pd
import numpy as np
#for text pre-processing
import re, string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
#for model-building
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.metrics import roc_curve, auc, roc_auc_score
# bag of words
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
#for word embedding
import gensim
from gensim.models import Word2Vec

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [6]:
#SPLITTING THE TRAINING DATASET INTO TRAIN AND TEST
X_train, X_test, y_train, y_test = train_test_split(df["Text"],df["target"],test_size=0.3,random_state = 0)
#Word2Vec
# Word2Vec runs on tokenized sentences
X_train_tok= [nltk.word_tokenize(i) for i in X_train]
X_test_tok= [nltk.word_tokenize(i) for i in X_test]

In [12]:
#Tf-Idf
tfidf_vectorizer = TfidfVectorizer(use_idf=True)
X_train_vectors_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_vectors_tfidf = tfidf_vectorizer.transform(X_test)

#Bow
bow_vectorizer = CountVectorizer()
X_train_vectors_bow = bow_vectorizer.fit_transform(X_train)
X_test_vectors_bow = bow_vectorizer.transform(X_test)
#building Word2Vec model
class MeanEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        # if a text is empty we should return a vector of zeros
        # with the same dimensionality as all the other vectors
        self.dim = len(next(iter(word2vec.values())))
    def fit(self, X, y):
        return self
    def transform(self, X):
        return np.array([
            np.mean([self.word2vec[w] for w in words if w in self.word2vec]
                    or [np.zeros(self.dim)], axis=0)
            for words in X
        ])
df['clean_text_tok']=[nltk.word_tokenize(i) for i in df['Text']]
model = Word2Vec(df['clean_text_tok'],min_count=1)
w2v = dict(zip(model.wv.index_to_key, model.wv.vectors))

modelw = MeanEmbeddingVectorizer(w2v)



# converting text to numerical data using Word2Vec
X_train_vectors_w2v = modelw.transform(X_train_tok)
X_val_vectors_w2v = modelw.transform(X_test_tok)

In [8]:
#FITTING THE CLASSIFICATION MODEL using SVM(W2v)
from sklearn import svm
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto',probability=True)



SVM.fit(X_train_vectors_w2v, y_train)

#lr_tfidf=LogisticRegression(solver = 'liblinear', C=10, penalty = 'l2')
#lr_tfidf.fit(X_train_vectors_tfidf, y_train)
#Predict y value for test dataset
y_predict = SVM.predict(X_val_vectors_w2v)
y_prob = SVM.predict_proba(X_val_vectors_w2v)[:,1]
print(classification_report(y_test,y_predict))
print('Confusion Matrix:',confusion_matrix(y_test, y_predict))

fpr, tpr, thresholds = roc_curve(y_test, y_prob)
roc_auc = auc(fpr, tpr)
print('AUC:', roc_auc)

              precision    recall  f1-score   support

           0       0.88      1.00      0.93       280
           1       0.00      0.00      0.00        39

    accuracy                           0.88       319
   macro avg       0.44      0.50      0.47       319
weighted avg       0.77      0.88      0.82       319

Confusion Matrix: [[280   0]
 [ 39   0]]
AUC: 0.6293040293040294


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [9]:
#FITTING THE CLASSIFICATION MODEL using Naive Bayes(tf-idf)
nb_tfidf = MultinomialNB()
nb_tfidf.fit(X_train_vectors_tfidf, y_train)
#Predict y value for test dataset
y_predict = nb_tfidf.predict(X_test_vectors_tfidf)
y_prob = nb_tfidf.predict_proba(X_test_vectors_tfidf)[:,1]
print(classification_report(y_test,y_predict))
print('Confusion Matrix:',confusion_matrix(y_test, y_predict))

fpr, tpr, thresholds = roc_curve(y_test, y_prob)
roc_auc = auc(fpr, tpr)
print('AUC:', roc_auc)

              precision    recall  f1-score   support

           0       0.88      1.00      0.93       280
           1       0.00      0.00      0.00        39

    accuracy                           0.88       319
   macro avg       0.44      0.50      0.47       319
weighted avg       0.77      0.88      0.82       319

Confusion Matrix: [[280   0]
 [ 39   0]]
AUC: 0.7753663003663003


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [10]:
#FITTING THE CLASSIFICATION MODEL using SVM(tf-idf)
from sklearn import svm
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto',probability=True)



SVM.fit(X_train_vectors_tfidf, y_train)

#lr_tfidf=LogisticRegression(solver = 'liblinear', C=10, penalty = 'l2')
#lr_tfidf.fit(X_train_vectors_tfidf, y_train)
#Predict y value for test dataset
y_predict = SVM.predict(X_test_vectors_tfidf)
y_prob = SVM.predict_proba(X_test_vectors_tfidf)[:,1]
print(classification_report(y_test,y_predict))
print('Confusion Matrix:',confusion_matrix(y_test, y_predict))

fpr, tpr, thresholds = roc_curve(y_test, y_prob)
roc_auc = auc(fpr, tpr)
print('AUC:', roc_auc)

              precision    recall  f1-score   support

           0       0.93      1.00      0.96       280
           1       0.95      0.49      0.64        39

    accuracy                           0.93       319
   macro avg       0.94      0.74      0.80       319
weighted avg       0.94      0.93      0.92       319

Confusion Matrix: [[279   1]
 [ 20  19]]
AUC: 0.946062271062271


In [13]:
#FITTING THE CLASSIFICATION MODEL using SVM(bow)
from sklearn import svm
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto',probability=True)



SVM.fit(X_train_vectors_bow, y_train)

#lr_tfidf=LogisticRegression(solver = 'liblinear', C=10, penalty = 'l2')
#lr_tfidf.fit(X_train_vectors_tfidf, y_train)
#Predict y value for test dataset
y_predict = SVM.predict(X_test_vectors_bow)
y_prob = SVM.predict_proba(X_test_vectors_bow)[:,1]
print(classification_report(y_test,y_predict))
print('Confusion Matrix:',confusion_matrix(y_test, y_predict))

fpr, tpr, thresholds = roc_curve(y_test, y_prob)
roc_auc = auc(fpr, tpr)
print('AUC:', roc_auc)

              precision    recall  f1-score   support

           0       0.95      0.98      0.96       280
           1       0.82      0.59      0.69        39

    accuracy                           0.93       319
   macro avg       0.88      0.79      0.82       319
weighted avg       0.93      0.93      0.93       319

Confusion Matrix: [[275   5]
 [ 16  23]]
AUC: 0.9427655677655679


In [ ]:
df_test = pd.read_csv('CleanedData2.csv')
X_test=df_test['Malay']
#converting words to numerical data using tf-idf
X_vector=tfidf_vectorizer.transform(X_test)
#use the best model to predict 'target' value for the new dataset
y_predict = SVM.predict(X_vector)
y_prob = SVM.predict_proba(X_vector)[:,1]
df_test['predict_prob']= y_prob
df_test['target']= y_predict
final=df_test[['Malay','target']].reset_index(drop=True)
final

,Malay,target
0,polis tangkap,0
1,mengapakah lokasi kebakaran terlalu spesifik,0
2,Saya menyesal kerana saya ingin menyambut hari...,0
3,meriah tah,0
4,asalkan dapat habis kerja kat cik kl br jemput...,0
5,"hmmm nak pasang pencawang telefon, asyik buat ...",0
6,yang tinggal hanyalah menyahpasang shopee,0
7,juta,0
8,assalamualaikum yb adakah anda akan turun ke p...,0
